In [2]:
import numpy as np

class BM25Fe():
    """Implements the BM25F scoring algorithm.
    """
 
    def __init__(self, K1=1.2, d_B=(0.75, 0.75), d_W = (1, 1), K3=1.2, q_B=(0.75, 0.75), q_W=(1, 1)):
        """
        
        :param B: free parameter, see the BM25 literature. Keyword arguments of
            the form ``fieldname_B`` (for example, ``body_B``) set field-
            specific values for B.
        :param K1: free parameter, see the BM25 literature.
        """
        
        self.K1 = K1
        self.d_B = d_B
        self.d_W = d_W
        
        self.K3 = K3
        self.q_B = q_B
        self.q_W = q_W
        
        self.fields = len(d_B)
        
        
    def score(self, idf, avgfl, doc, query):
        
        doc = numpy.array(doc)
        query = numpy.array(query)
        
        doc_n = numpy.zeros(doc[0].size)
        query_n = numpy.zeros(query[0].size)
        for i in xrange(self.fields):
            doc_n += self.d_W[i]/(1-self.d_B[i]+self.d_B[i]*doc[i].sum()/avgfl[i])*doc[i]
            query_n += self.q_W[i]/(1-self.q_B[i]+self.q_B[i]*query[i].sum()/avgfl[i])*query[i]
 
        return (idf*(K1+1)*doc_n/(K1+doc_n)*(K3+1)*query_n/(K3+query_n)).sum()